# scikit-learnのトレーニング♨

## RNN 編
- [環境準備](#環境準備)
- [RNNアルゴリズム・モデル](#RNNアルゴリズム・モデル)
  - [時系列予測](#時系列予測)
  - [...](#...)

## [目次](TableOfContents.ipynb)

## 参考
開発基盤部会 Wiki
- データマイニング（DM）- Python - DL  
https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3%E3%82%B0%EF%BC%88DM%EF%BC%89-%20Python%20-%20DL

## 環境準備

In [ ]:
import io
import requests

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.metrics import confusion_matrix as cm
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
print(tf.__version__)

import keras
print(keras.__version__)
# モデル定義
from keras.models import Model, Sequential, model_from_json
from keras.layers import Dense, Input, Activation, Flatten, Dropout, LSTM
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
from keras.optimizers import SGD, Adam
# その他
from keras.applications.vgg16 import VGG16
from keras.utils import np_utils

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## RNNアルゴリズム・モデル

### 時系列予測

#### データ

##### 生成

In [ ]:
proxies = { # プロキシ設定
"http":"http://<user_name>:<password>@<proxy_host>:<proxy_port>/",
"https":"https://<user_name>:<password>@<proxy_host>:<proxy_port>/"
}

url = 'https://raw.githubusercontent.com/AileenNielsen/TimeSeriesAnalysisWithPython/master/data/AirPassengers.csv'
res = requests.get(url) # , verify=False, proxies=proxies) # プロキシ
df = pd.read_csv(io.BytesIO(res.content), encoding='shift-jis', sep=",")
df

##### 加工

##### 理解

###### 先頭

In [ ]:
df.head()

###### 後尾

In [ ]:
df.tail()

###### 列名変更

In [ ]:
df.columns = ['Month', 'Passengers']

###### 可視化

In [ ]:
plt.plot(df['Passengers'])
plt.xticks(np.arange(0, 145, 12))
plt.grid()
plt.show()

###### 基本成分に分解
グラフの上から
- Observed : 原系列  
元データのプロット
- Trend : 傾向変動  
全体的な変化の傾向
- seasonal : 季節変動  
周期的に繰り返す変動
- residual : 不規則変動 (残差)  
トレンドと季節性を除いたその他変動成分

に分解する（[参考](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88%E3%83%AF%E3%83%BC%E3%82%AF#y11573fa)）。

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
sd = seasonal_decompose(df['Passengers'].values, period=12) # periodで周期を指定
sd.plot()
plt.show()

##### 準備

###### 型変換
Kerasが扱える型に変換

In [ ]:
data = df['Passengers'].values.astype('f')
data

###### 正規化
- しないと上手く学習できない。
- X・Yともに正規化するので、推論結果はscaleを掛けて戻す。

In [ ]:
scale = data.max()
data /= scale

###### 説明系列と目的系列、訓練データの作成
このケースでは説明系列 ≒ 目的系列（-> [詳細はコチラ](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88%E3%83%AF%E3%83%BC%E3%82%AF#pafaec18)）

In [ ]:
# dataは、0-143
x = data[:-1] # 0-142
y = data[1:]  # 1-143
print('x:',len(x))
print('y:', len(y))
print('x:', x[:5])
print('y:', y[:5])

###### shape変換
Kerasの時系列解析用にshape変換
- x : 説明系列の配列（複数の説明系列）の配列（バッチ）の可能性で３次元
- y : 目的系列の配列（バッチ）の可能性で２次元

In [ ]:
print('x:', np.shape(x), ' y:', np.shape(y))
x = x.reshape(len(x), 1, 1)
y = y.reshape(len(y), 1)
print('x:', np.shape(x), ' y:', np.shape(y))
print('x:', x[:3])
print('y:', y[:3])

###### 分割
時系列を維持して訓練・テストのデータ分割（-> [詳細はコチラ](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88%E3%83%AF%E3%83%BC%E3%82%AF#af105024)）

In [ ]:
# 訓練データのサンプル数を指定
train_size = int(len(data) * 0.7)
# データの分割
x_train = x[:train_size]
x_test = x[train_size:]
y_train = y[:train_size]
y_test = y[train_size:]
# shapeを確認
print('x_train:', x_train.shape)
print('x_test :', x_test.shape)
print('y_train:', y_train.shape)
print('y_test :', y_test.shape)

#### モデリング

##### LSTMの定義
- 30ユニットのLSTMの層 ＋ Denseレイヤ = 1つの値を予測
- LSTM（units, batch_input_shape=batch_size, time_step, input_dim））
  - units: 中間層のノード数（中間層の出力次元数）
  - batch_input_shape: 入力するデータの形状を指定
    - バッチサイズ
    - 予測のタイムステップ
    - 入力の次元（特徴量（説明系列）の数）

In [ ]:
model = Sequential()
model.add(LSTM(30, batch_input_shape=(None, 1, 1))) # 中間層が30のLSTM
model.add(Dense(1)) # 回帰なので最後の出力値は１つ

##### コンパイル
- 回帰の損失関数は誤差二乗和（mse ≒ mean_squared_error）
- [optimizer=Adam](TensorFlowAndKeras0.ipynb)を指定する。

In [ ]:
model.compile(loss='mean_squared_error', optimizer=Adam())

##### 確認

In [ ]:
model.summary()

##### 実行

###### 学習

In [ ]:
batch_size = 20
n_epoch = 200
hist = model.fit(x_train,
                 y_train,
                 epochs=n_epoch,
                 validation_data=(x_test, y_test),
                 verbose=0,
                 batch_size=batch_size)

###### 推論

In [ ]:
y_pred = model.predict(x)

In [ ]:
# 出力の正規化を戻す関数
def pred_n_passengers(y_pred, scale, year, month):
    index = ((year - 1949) * 12) + (month - 1) # 1949/1からのデータ
    return y_pred[index] * scale # 正規化した値を元に戻す。

In [ ]:
# 試しに､1960年4月の乗客数を予測
year = 1960
month = 4
print("org : ", data[((year - 1949) * 12) + (month - 1)] * scale)
print("pred: ", pred_n_passengers(y_pred, scale, year, month))

##### 評価
回帰なので正答率は出力しない。

###### 実測・予測を表示

In [ ]:
plt.plot(data, color='blue')  # 実測値
plt.plot(y_pred, color='red')   # 予測値
plt.show()

###### 学習履歴を表示

In [ ]:
def plot_history_loss(hist):
    plt.plot(hist.history['loss'],label="loss for training")
    plt.plot(hist.history['val_loss'],label="loss for validation")
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='best')
    plt.show()

In [ ]:
plot_history_loss(hist)

### ...